*Created On: Feb 9, 2026*
## Description
To prototype a `EHRFoundationalModelMIMIC4(BaseTask)` class, focusing first on clinical discharge notes + radiology notes pre-processor

### Setup

In [1]:
# Change directory to package root
import os
PROJECT_ROOT = '/Users/wpang/Desktop/PyHealth'
os.chdir(PROJECT_ROOT)

# Other General Packages
from datetime import datetime
from typing import Any, Dict, List, Optional

In [2]:
# PyHealth Packages
from pyhealth.datasets import MIMIC4Dataset
from pyhealth.tasks import MultimodalMortalityPredictionMIMIC4

In [3]:
import polars as pl

pl.Config.set_tbl_rows(1000)
pl.Config.set_tbl_cols(100)
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [4]:
# Paths
EHR_ROOT = os.path.join(PROJECT_ROOT, "srv/local/data/physionet.org/files/mimiciv/2.2")
NOTE_ROOT = os.path.join(PROJECT_ROOT, "srv/local/data/physionet.org/files/mimic-iv-note/2.2")
CXR_ROOT = os.path.join(PROJECT_ROOT,"srv/local/data/physionet.org/files/mimic-cxr-jpg/2.0.0")
CACHE_DIR = os.path.join(PROJECT_ROOT,"srv/local/data/wp/pyhealth_cache")

In [5]:
class EHRFoundationalModelMIMIC4:
    def __init__(self):
        """Initialize the EHR Foundational Model task."""
        self.input_schema: Dict[str, str] = {
            "discharge": "raw",
            "radiology": "raw"
        }
        self.output_schema: Dict[str, str] = {"mortality": "binary"}

    def _clean_text(self, text: Optional[str]) -> Optional[str]:
        """Return text if non-empty, otherwise None."""
        return text if text else None

    def __call__(self, patient: Any) -> List[Dict[str, Any]]:
          all_discharge_notes = []
          all_radiology_notes = []

          # Get all discharge notes for this patient
          discharge_notes = patient.get_events(event_type="discharge")
          for note in discharge_notes:
              try:
                  note_text = self._clean_text(note.text)
                  if note_text:
                      all_discharge_notes.append(note_text)
              except AttributeError:
                  pass

          # Get all radiology notes for this patient
          radiology_notes = patient.get_events(event_type="radiology")
          for note in radiology_notes:
              try:
                  note_text = self._clean_text(note.text)
                  if note_text:
                      all_radiology_notes.append(note_text)
              except AttributeError:
                  pass

          # Require at least one note of either type
          if len(all_discharge_notes) == 0 and len(all_radiology_notes) == 0:
              return []

          return [
              {
                  "patient_id": patient.patient_id,
                  "discharge": all_discharge_notes,
                  "radiology": all_radiology_notes
              }
          ]

### Utilities

In [6]:
import shutil
def delete_cache(cache_directory):
    for item in os.listdir(cache_directory):
        item_path = os.path.join(cache_directory, item)
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

    print(f"Cache deleted successfully from: {cache_directory}")

delete_cache(CACHE_DIR)

Cache deleted successfully from: /Users/wpang/Desktop/PyHealth/srv/local/data/wp/pyhealth_cache


### Code

In [7]:
dataset = MIMIC4Dataset(
        note_root=NOTE_ROOT,
        note_tables=["discharge", "radiology"],
        cache_dir=CACHE_DIR,
        num_workers=16
    )

# To Display:
# dataset.global_event_df.head(10).collect().to_pandas()

Memory usage Starting MIMIC4Dataset init: 450.2 MB
Initializing mimic4 dataset from None|/Users/wpang/Desktop/PyHealth/srv/local/data/physionet.org/files/mimic-iv-note/2.2|None (dev mode: False)
Initializing MIMIC4NoteDataset with tables: ['discharge', 'radiology'] (dev mode: False)
Using default note config: /Users/wpang/Desktop/PyHealth/pyhealth/datasets/configs/mimic4_note.yaml
Memory usage Before initializing mimic4_note: 450.2 MB
Initializing mimic4_note dataset from /Users/wpang/Desktop/PyHealth/srv/local/data/physionet.org/files/mimic-iv-note/2.2 (dev mode: False)
Memory usage After initializing mimic4_note: 450.3 MB
Memory usage After Note dataset initialization: 450.3 MB
Memory usage Completed MIMIC4Dataset init: 450.3 MB


/Users/wpang/Desktop/PyHealth/pyhealth/datasets/mimic4.py:103: UserWarning: Events from discharge table only have date timestamp (no specific time). This may affect temporal ordering of events.
  warnings.warn(


In [8]:
task = EHRFoundationalModelMIMIC4()                                                                                   
patient = dataset.get_patient("10000032")
samples = task(patient)                                                                                               

Combining data from note dataset
Scanning table: discharge from /Users/wpang/Desktop/PyHealth/srv/local/data/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv.gz
Scanning table: radiology from /Users/wpang/Desktop/PyHealth/srv/local/data/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv.gz
Creating combined dataframe
Caching event dataframe to /Users/wpang/Desktop/PyHealth/srv/local/data/wp/pyhealth_cache/global_event_df.parquet...
Found 43 unique patient IDs


In [10]:
print(samples)

[{'patient_id': '10000032', 'discharge': [' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. Sh